In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-03-04 12:23:35,635 INFO: Initializing external client
2025-03-04 12:23:35,636 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:23:36,450 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Fetching data from 2025-02-03 17:23:35.634452+00:00 to 2025-03-04 16:23:35.634452+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.54s) 


In [4]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 18:00:00
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-03 18:00:00
2,0,1,3,3,6,2,3,6,2,0,...,3,5,1,2,2,3,0,1,4,2025-03-03 18:00:00
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-03 18:00:00
4,1,5,1,2,0,0,0,0,1,0,...,3,2,8,3,4,1,3,4,7,2025-03-03 18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,259,2025-03-03 18:00:00
247,1,3,0,1,0,0,0,0,0,0,...,0,1,4,3,2,5,1,6,260,2025-03-03 18:00:00
248,41,24,17,16,11,3,1,3,0,0,...,7,16,25,36,32,26,33,43,261,2025-03-03 18:00:00
249,73,43,34,22,4,6,1,3,1,1,...,118,98,79,71,82,97,87,82,262,2025-03-03 18:00:00


In [5]:
predictions = fetch_next_hour_predictions()

2025-03-04 12:23:46,072 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 12:23:46,074 INFO: Initializing external client
2025-03-04 12:23:46,074 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 12:23:46,750 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215680
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 
Current UTC time: 2025-03-04 17:23:46.072722+00:00
Next hour: 2025-03-04 18:00:00+00:00
Found 251 records


In [6]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
251,14,1.0,2025-03-04 18:00:00+00:00
252,70,20.0,2025-03-04 18:00:00+00:00
253,200,0.0,2025-03-04 18:00:00+00:00
254,101,0.0,2025-03-04 18:00:00+00:00
255,130,1.0,2025-03-04 18:00:00+00:00
...,...,...,...
497,142,10.0,2025-03-04 18:00:00+00:00
498,41,5.0,2025-03-04 18:00:00+00:00
499,226,2.0,2025-03-04 18:00:00+00:00
500,190,0.0,2025-03-04 18:00:00+00:00


In [7]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [8]:
top10

[236, 237, 132, 161, 138, 164, 239, 186, 107, 238]

In [9]:
top10

[236, 237, 132, 161, 138, 164, 239, 186, 107, 238]

In [10]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
246    False
247    False
248    False
249    False
250    False
Name: pickup_location_id, Length: 251, dtype: bool

In [11]:
features_cp = features.copy()

In [12]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [13]:
historical_df

,datetime,rides
0,2025-02-03 18:00:00,484.0
1,2025-02-03 19:00:00,395.0
2,2025-02-03 20:00:00,364.0
3,2025-02-03 21:00:00,283.0
4,2025-02-03 22:00:00,137.0
...,...,...
668,2025-03-03 14:00:00,327.0
669,2025-03-03 15:00:00,323.0
670,2025-03-03 16:00:00,372.0
671,2025-03-03 17:00:00,501.0
